In [ ]:
#04/13/2024
#CIS 3120 HW#3 Part 1: Web Scraping Tables and Inputting it into the API

#Importing all the necessary libraires.
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import numpy as np

#The function below will take the parameter of the url of the website that we want to scrape and output all the top 100
#publicly traded companies symbols in a dataframe for our api requests later. However, we would also be getting the company name too.
#I decided not to scrap the company market cap and price from the site because we can just get them from the api later.
def top_100_publicly_traded_companies_ticker(url):
    #Lists for the company name and ticker that we wil get later.
    company_name = []
    ticker = []
    page = requests.get(url) #Making the request to the url
    if page.status_code == 200: #If the page status code is 200, we can start scraping, if not it will output an error message.
        soup = BeautifulSoup(page.content, "html.parser") #Importing the raw html into beautiful soup.
        company_tags = soup.find_all('td', class_ = 'name-td') #Finding all the td tables with the class of "name-td" into a list.
        #The for loop below would take all the td tags and find the first instance of the div with the class of either company name or
        #company code and assign them to a variable. Afterwards, it will append each of the ticker and name to the variables.
        for company in company_tags:
            companies = company.find('div', class_ = 'company-name') #Getting the company name
            tickers = company.find('div', class_ = 'company-code') #Getting the company code
            if '.' not in tickers.get_text(): #This if statement is to tell the scrapper to see if there is another period on the ticker because the API can't seem to find the information for some international companies.
                ticker.append(tickers.get_text())
                company_name.append(companies.get_text())

        #Making the df1 dataframe using pandas with the information that we scraped earlier
        df1 = pd.DataFrame({
                'Company Name' : company_name,
                'Company Ticker' : ticker
            })
        return df1 #Returning the dataframe
    else:
        print(f"This website is not available. The status code is {page.status_code}.")

#The function below would take the arguement of the df1['Company Ticker'] tocall the Real-Time Finance Data API from the website RAPIDAPI.
#It would return another dataframe called df2 that contain the company stock's price, previous close price, change in price, year low price,
#year high price, CDP score, PE Ratio, and Market Cap.
#WARNING: THIS API CAN ONLY TAKE 200 REQUESTS PER MONTH. USE SPARELY!!!
def realtime_finance_api(tickers):

    #List of the information that we want from the company.
    price = []
    previous_close = []
    change = []
    year_low = []
    year_high = []
    cdp_score = []
    pe_ratio = []
    market_cap = []

    #A for loop so that for each of the tickers in the df1['Company Ticker'], we would get each information seperately.
    for ticker in tickers:

        #Making the calls to the API
        url = "https://real-time-finance-data.p.rapidapi.com/stock-overview"

        querystring = {f"symbol":{ticker},"language":"en"} #The Query to get the information from the API

        headers = {
            "X-RapidAPI-Key": "Insert here",
            "X-RapidAPI-Host": "real-time-finance-data.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)

        if response.status_code == 200: #Making sure that we get a valid response from the API
            data = response.json() #Getting the API data in json format
            #We appending most of the data in float form.
            price.append(float(data['data']['price'])) #Appending the price of that company stock to the list by going to the data key in the series and getting the price key from the data key.
            previous_close.append(float(data['data']['previous_close'])) #Appending the previous close price of that company stock to the list by going to the data key in the series and getting the previous close price key from the data key.
            change.append(float(data['data']['change'])) #Appending the change in price of that company stock to the list by going to the data key in the series and getting the change in price key from the data key.
            year_high.append(float(data['data']['year_high'])) #Appending the year high of that company stock to the list by going to the data key in the series and getting the year_high key from the data key.
            year_low.append(float(data['data']['year_low'])) #Appending the year low of that company stock to the list by going to the data key in the series and getting the year_low key from the data key.
            market_cap.append(round(float(data['data']['company_market_cap']), 2)) #Appending the market_cap of that company to the list by going to the data key in the series and getting the market_cap key from the data key. We will round it to the hundredth place.

            #The code below will try to scrape the company's cdp score, if it is not available, we would say it is not available.
            try:
                cdp_score.append(data['data']['company_cdp_score']) #Appending the cdp score of that company to the list by going to the data key in the series and getting the company_cdp_score key from the data key.
            except:
                cdp_score.append('Not available')

            #The code below will try to scrape the company's pe, if it is not available, we would append nan to it.
            try:
                pe_ratio.append(round(float(data['data']['company_pe_ratio']), 2)) #Appending the pe ratio of that company stock to the list by going to the data key in the series and getting the company_pe_ratio key from the data key. We will round it to the hundredth place.
            except:
                pe_ratio.append(np.nan)
        else:
            print(f"API request was unsuccessful. The status code is {response.status_code}.")

    #Making the second dataframe on the stock of the 100 companies that we scraped.
    df2 = pd.DataFrame({
        'Last Price (USD)' : price,
        'Previous Closing Price (USD)' : previous_close,
        'Change in Price' : change,
        'Year Low' : year_low,
        'Year High' : year_high,
        'Carbon Disclosure Rating Score' : cdp_score,
        'PE Ratio' : pe_ratio,
        'Market Capitalization (USD)' : market_cap
    })

    return df2 #Returning the dataframe

#The main function will be used to insert the website that we will use to be scraped, assigning the dataframes names, and merging them horizontally
#To a new dataframe called df3. We will then output df3 as a csv file, print the dataframe, and the min, max, etc of each
#numerical column of the dataframe.
def main():
    url = 'https://companiesmarketcap.com/' #Website that we will be using
    df1 = top_100_publicly_traded_companies_ticker(url) #Assigning the first dataframe
    df2 = realtime_finance_api(df1['Company Ticker']) #Assigning the second dataframe by passing through the ticker names of each companies into the API.
    df3 = pd.concat([df1, df2], axis = 1) #Mergeing the dataframes horizontally.

    df3.to_csv('Top Publicly Traded Companies Stocks By Market Cap.csv', index=False) #Outputting the csv file and getting rid of the indexes.
    print(df3) #Displaying the dataframe
    print(df3.describe().apply(lambda s: s.apply('{0:.2f}'.format))) #Displaying the  statistics for the combined dataframe.

main()

#Please see below for a cleaner describe output.

            Company Name Company Ticker  Last Price (USD)  \
0              Microsoft           MSFT            421.90   
1                  Apple           AAPL            176.55   
2                 NVIDIA           NVDA            881.86   
3      Alphabet (Google)           GOOG            159.19   
4                 Amazon           AMZN            186.13   
..                   ...            ...               ...   
74        Morgan Stanley             MS             86.19   
75  Royal Bank Of Canada             RY            135.66   
76                  Nike            NKE             92.00   
77         Philip Morris             PM             88.85   
78    Intuitive Surgical           ISRG            384.80   

    Previous Closing Price (USD)  Change in Price  Year Low  Year High  \
0                         427.93            -6.03    275.37    430.820   
1                         175.04             1.51    161.42    199.620   
2                         906.16           -2

In [ ]:
#The first describe that I outputted had euler numbers, but I can't run the code again or else I would reach my API limit
#For the month, so I did this to make the statistics cleaner

df3 = pd.read_csv('https://raw.githubusercontent.com/alvliu03/Company_Stock_Price_By_Capitalization/main/Top%20Publicly%20Traded%20Companies%20Stocks%20By%20Market%20Cap.csv') #Reading the outputted csv file
print(df3.describe().apply(lambda s: s.apply('{0:.2f}'.format))) #lambda s: creates an anonymous function that takes an argument s. s.apply('{0:.2f}'.format) applies the '{0:.2f}'.format function to each element in s. This function formats a floating-point number to a string with two decimal places.

      Last Price (USD) Previous Closing Price (USD) Change in Price Year Low  \
count            79.00                        79.00           79.00    79.00   
mean            245.78                       249.37           -3.59   161.82   
std             243.89                       247.85            5.49   132.30   
min              23.05                        23.32          -38.39    16.39   
25%              87.52                        88.14           -4.59    68.03   
50%             162.28                       165.97           -1.79   113.10   
75%             304.86                       312.17           -0.61   213.93   
max            1344.07                      1382.46            1.51   601.29   

      Year High PE Ratio Market Capitalization (USD)  
count     79.00    75.00                       79.00  
mean     271.22    49.33             450160450165.63  
std      262.00   104.67             629395238380.74  
min       25.72     0.01             135137934395.78  
25% 

In [ ]:
#Sample API Output

#Example company is Apple or aapl

# {
#   "status": "OK",
#   "request_id": "0ce198af-5501-41f8-bc9f-58e7d2add71d",
#   "data": {
#     "symbol": "AAPL:NASDAQ",
#     "name": "Apple Inc",
#     "type": "stock",
#     "price": 151.29,
#     "open": 152.305,
#     "high": 152.7,
#     "low": 149.97,
#     "volume": 74829573,
#     "previous_close": 150.72,
#     "change": 0.57,
#     "change_percent": 0.3782,
#     "pre_or_post_market": 150.88,
#     "pre_or_post_market_change": -0.41,
#     "pre_or_post_market_change_percent": -0.271,
#     "last_update_utc": "2022-11-19 01:00:00",
#     "country_code": "US",
#     "exchange": "NASDAQ",
#     "exchange_open": "2022-11-18 09:30:00",
#     "exchange_close": "2022-11-18 16:00:00",
#     "timezone": "America/New_York",
#     "utc_offset_sec": -18000,
#     "currency": "USD",
#     "about": "Apple Inc. is an American multinational technology company headquartered in Cupertino, California, United States. Apple is the largest technology company by revenue and, as of June 2022, is the world's biggest company by market capitalization, the fourth-largest personal computer vendor by unit sales and second-largest mobile phone manufacturer. It is one of the Big Five American information technology companies, alongside Alphabet, Amazon, Meta, and Microsoft.\nApple was founded as Apple Computer Company on April 1, 1976, by Steve Jobs, Steve Wozniak and Ronald Wayne to develop and sell Wozniak's Apple I personal computer. It was incorporated by Jobs and Wozniak as Apple Computer, Inc. in 1977 and the company's next computer, the Apple II, became a best seller and one of the first mass-produced microcomputers. Apple went public in 1980 to instant financial success. The company developed computers featuring innovative graphical user interfaces, including the 1984 original Macintosh, announced that year in a critically acclaimed advertisement. By 1985, the high cost of its products and power struggles between executives caused problems.",
#     "year_low": 129.04,
#     "year_high": 182.94,
#     "primary_exchange": "NASDAQ",
#     "company_website": "http://www.apple.com/",
#     "company_country_code": "US",
#     "company_country": "United States",
#     "company_state": "California",
#     "company_city": "Cupertino",
#     "company_street_address": "One Apple Park Way",
#     "company_ceo": "Tim Cook",
#     "company_employees": 164000,
#     "company_cdp_score": "A-",
#     "company_founded_date": "1976-04-01",
#     "company_cdp_url": "https://www.cdp.net/en/responses/865",
#     "avg_volume": 89998949,
#     "company_pe_ratio": 24.7481,
#     "company_market_cap": 2406739367994.995,
#     "company_dividend_yield": 0.6081,
#     "wikipedia_url": "https://en.wikipedia.org/wiki/Apple_Inc.",
#     "google_mid": "/m/07zmbvf"
#   }
# }